In [3]:
import string
from hashlib import sha1
from datetime import datetime
import pickle
import ast
from jinja2 import Environment, FileSystemLoader,Template
import pandas as pd
import numpy as np
from tqdm import tqdm

data=pd.read_excel("C:/Users/bharg/Downloads/masterwiki.xlsx")
company = data.copy()
company.drop(["Unnamed: 0"],axis=1,inplace=True)
tewiki = '''<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="te">
	<siteinfo>
		<sitename>tewiki</sitename>
		<dbname>indicwiki</dbname>
		<base>https://tewiki.iiit.ac.in/index.php/%E0%B0%AE%E0%B1%8A%E0%B0%A6%E0%B0%9F%E0%B0%BF_%E0%B0%AA%E0%B1%87%E0%B0%9C%E0%B1%80</base>
		<generator>MediaWiki 1.34.0</generator>
		<case>first-letter</case>
		<namespaces>
			<namespace key="-2" case="first-letter">మీడియా</namespace>
			<namespace key="-1" case="first-letter">ప్రత్యేక</namespace>
			<namespace key="0" case="first-letter" />
			<namespace key="1" case="first-letter">చర్చ</namespace>
			<namespace key="2" case="first-letter">వాడుకరి</namespace>
			<namespace key="3" case="first-letter">వాడుకరి చర్చ</namespace>
			<namespace key="4" case="first-letter">Project</namespace>
			<namespace key="5" case="first-letter">Project చర్చ</namespace>
			<namespace key="6" case="first-letter">దస్త్రం</namespace>
			<namespace key="7" case="first-letter">దస్త్రంపై చర్చ</namespace>
			<namespace key="8" case="first-letter">మీడియావికీ</namespace>
			<namespace key="9" case="first-letter">మీడియావికీ చర్చ</namespace>
			<namespace key="10" case="first-letter">మూస</namespace>
			<namespace key="11" case="first-letter">మూస చర్చ</namespace>
			<namespace key="12" case="first-letter">సహాయం</namespace>
			<namespace key="13" case="first-letter">సహాయం చర్చ</namespace>
			<namespace key="14" case="first-letter">వర్గం</namespace>
			<namespace key="15" case="first-letter">వర్గం చర్చ</namespace>
			<namespace key="828" case="first-letter">మాడ్యూల్</namespace>
			<namespace key="829" case="first-letter">మాడ్యూల్ చర్చ</namespace>
			<namespace key="2300" case="first-letter">Gadget</namespace>
			<namespace key="2301" case="first-letter">Gadget talk</namespace>
			<namespace key="2302" case="case-sensitive">Gadget definition</namespace>
			<namespace key="2303" case="case-sensitive">Gadget definition talk</namespace>
			<namespace key="2600" case="first-letter">Topic</namespace>
		</namespaces>
	</siteinfo>'''


# 717000 -  => companies's articles

page_id = 722001

user_id ="18869"
username ="Bhargav"

# Funtions to write page to file object
def sha36(page_id):
	page_id = str(page_id).encode('utf-8')
	sha16 =sha1(page_id).hexdigest()
	sha10 =int(sha16, 16)

	chars =[]
	alphabets = string.digits +string.ascii_lowercase
	while sha10>0:
		sha10, r = divmod(sha10, 36)
		chars.append(alphabets[r])
	
	return ''.join(reversed(chars))

# Function to replace possible Entity references
def clean(text):
	text = text.replace('&',"&amp;")
	text = text.replace('<',"&lt;")
	text = text.replace('>',"&gt;")
	text = text.replace('"',"&quot;")
	text = text.replace("'","&apos;")

	return text

def writePage(title, wikiText, fobj):
	global user_id, username, page_id

	pglen = len(wikiText)
	time =datetime.now().strftime("%Y-%m-%dT%H-%M-%SZ")
	
	curPage ='''\n\n
	<page>
		<title>''' +clean(title) +'''</title>
		<ns>0</ns>
		<id>''' +str(page_id) +'''</id>
		<revision>
			<id>''' +str(page_id) +'''</id>
			<timestamp>'''+time+'''</timestamp>
			<contributor>
				<username>''' +username +'''</username>
				<id>''' +str(user_id) +'''</id>
			</contributor>
			<comment>xmlpage created</comment>
			<model>wikitext</model>
			<format>text/x-wiki</format>
			<text xml:space="preserve" bytes="''' +str(pglen) +'''">
			\n''' + clean(wikiText) +'''
			</text>
			<sha1>''' +sha36(page_id) +'''</sha1>
		</revision>
	</page>
	\n\n'''

	fobj.write(curPage)
	page_id += 1
	return


def get_Data(row,data):
    web1=str(row[15]).strip()
    web2=str(row[6]).strip()
    if web1!="-" and web1[0]!="h":
        web1="https://"+web1
    if web2!="-" and web2[0]!="h":
        web2="https://"+web2
    intro_data={
        "CIN":str(row[0]).strip(),
        "company_name":str(row[1]).strip(),
        "industry":str(row[2]).strip(),
        "sector":str(row[3]).strip(),
        "founded":str(int(row[5])).strip(),
        "key_people":str(row[8]).strip(),
        "headquarters":str(row[4]).strip(),
        "status":str(row[23]).strip(),
        "authorised_capital":str(row[32]).strip(),
        "paidup_capital":str(row[33]).strip(),
        "RoC":str(row[24]).strip(),
        "type":str(row[7]).strip(),
        "class_of_company":str(row[28]).strip(),
        "regno":str(row[25]).strip(),
        "wiki_history":str(row[39]).strip(),
        "wiki_operations":str(row[43]).strip(),
        "wiki_products":str(row[40]).strip(),
        "wiki_services":str(row[41]).strip(),
        "service":str(row[9]).strip(),
        "product":str(row[18]).strip(),
        "wiki_awards":str(row[42]).strip(),
        "logo":str(row["Image"]).strip(),
        "website":(web1,web2)
    }
    data.update(intro_data)
    return data

files=FileSystemLoader('./')
env=Environment(loader=files)
env.globals.update(zip=zip)
template = env.get_template("introduction.j2")

fname="company_full.xml"
fobj = open(fname, 'w',encoding='utf-8')
# fobj = open('birds.xml', 'w')
fobj.write(tewiki)


# for i in tqdm(range(birds.shape[0])):
j=0
for i in tqdm(range(104)):
	data={}
	row=company.iloc[j]
	j+=1
	d=get_Data(row,data)
	text = template.render(d)
	text=text.replace(" మరియు",",")
	text=text.replace("యొక్క","తమ")
	title=str(row[-1]).strip()
	writePage(title, text, fobj)


fobj.write('</mediawiki>')
fobj.close()


100%|██████████| 5/5 [00:00<00:00, 1653.77it/s]
